In [4]:
import tkinter as tk
import numpy as np

In [5]:
class Perceptron:
    """ A single neuron with the sigmoid activate function
        Attributtes:
            inputs: The number of inputs in the perceptron, not cunting the bias
            bias: the bias term. By default it´s 1.0
    """
    
    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs +1 (for the bias)"""
        self.weights = np.random.rand(inputs+1)*2 -1
        self.bias = bias
        
    def run(self,x):
        """Run the perceptron. x is a python list with the input values."""
        x_sum = np.dot(np.append(x,self.bias),self.weights)
        #this calculates the product point of the inputs and the weights
        return self.sigmoid(x_sum)
    
    def set_weights(self,w_init):
        """Set the weights. w_init is a python list with the weights"""
        self.weights = np.array(w_init)

    
    def sigmoid(self,x):
        """Evaluate the sigmoid function for thw floating point input x"""
        return 1/(1 + np.exp(-x))
    

In [6]:

class MultiLayerPerceptron:
    """A multilayer perceptron class that uses the perceptron class above.
        Attributtes:
            layers: A python list with the number of elements per layer
            bias: The bias term. The same bias is used for all neurons
            eta: The learning rate"""
    def __init__(self,layers,bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters"""
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] #the list of all neurons
        self.values = [] #the list of all output values
        self.d= []
        
        for i in range(len(self.layers)):
            self.values.append([])
            self.d.append([])
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d[i] = [0.0 for j in range(self.layers[i])]
            if i>0:
                for j in range(self.layers[i]):
                    self.network[i].append(Perceptron(inputs=self.layers[i-1],bias=self.bias))
    
        self.network = np.array([np.array(x) for x in self.network],dtype=object)
        self.values=np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)
    
    def set_weights(self,w_init):
        """set the weights.
            w_init is a list of lists with the weights for all, but the input layer"""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j])
                
    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
            print()
            
    def run(self,x):
        """Feed a sample x into the multilayer perceptron"""
        x=np.array(x,dtype=object)
        self.values[0]=x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.values[i][j]= self.network[i][j].run(self.values[i-1])
        return self.values[-1]
        
    #Backpropagation method
    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorythm)."""
        x = np.array(x, dtype=object)
        y = np.array(y, dtype=object)

        #Step 1:  feed a sample to the network
        outputs = self.run(x)

        #Step 2: calculate the MSE
        error = (y-outputs)
        MSE = sum(error ** 2)/ self.layers[-1]

        #Step 3: Calculate the output error terms
        self.d[-1] = outputs * (1-outputs)*error

        #Step 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]):
                    fwd_error+= self.network[i+1][k].weights[h] * self.d[i+1][k]
                self.d[i][h]=self.values[i][h] * (1-self.values[i][h]) * fwd_error

        #step 5 & 6 : calculate the deltas and update the weights
        #iterates layers
        for i in range(1, len(self.network)):
            #iterates neurons
            for j in range(self.layers[i]):
                #iterates inputs
                for k in range(self.layers[i-1]+1):
                    if k == self.layers[i-1]:
                        delta = self.eta * self.d[i][j] *self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k]+=delta

        return MSE

In [13]:
sdrnn = MultiLayerPerceptron(layers=[7,7,7])  # MLP
tepochs = 0

def update_a(event):
    r = int(slider_a.get() * (255 - offset)) + offset
    g = offset - int(slider_a.get() * (offset))
    b = offset - int(slider_a.get() * (offset))
    slider_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_b(event):
    r = int(slider_b.get() * (255 - offset)) + offset
    g = offset - int(slider_b.get() * (offset))
    b = offset - int(slider_b.get() * (offset))
    slider_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_c(event):
    r = int(slider_c.get() * (255 - offset)) + offset
    g = offset - int(slider_c.get() * (offset))
    b = offset - int(slider_c.get() * (offset))
    slider_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_d(event):
    r = int(slider_d.get() * (255 - offset)) + offset
    g = offset - int(slider_d.get() * (offset))
    b = offset - int(slider_d.get() * (offset))
    slider_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_e(event):
    r = int(slider_e.get() * (255 - offset)) + offset
    g = offset - int(slider_e.get() * (offset))
    b = offset - int(slider_e.get() * (offset))
    slider_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_f(event):
    r = int(slider_f.get() * (255 - offset)) + offset
    g = offset - int(slider_f.get() * (offset))
    b = offset - int(slider_f.get() * (offset))
    slider_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_g(event):
    r = int(slider_g.get() * (255 - offset)) + offset
    g = offset - int(slider_g.get() * (offset))
    b = offset - int(slider_g.get() * (offset))
    slider_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_out_a(event):
    r = int(out_a.get() * (255 - offset)) + offset
    g = offset - int(out_a.get() * (offset))
    b = offset - int(out_a.get() * (offset))
    out_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_b(event):
    r = int(out_b.get() * (255 - offset)) + offset
    g = offset - int(out_b.get() * (offset))
    b = offset - int(out_b.get() * (offset))
    out_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_c(event):
    r = int(out_c.get() * (255 - offset)) + offset
    g = offset - int(out_c.get() * (offset))
    b = offset - int(out_c.get() * (offset))
    out_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_d(event):
    r = int(out_d.get() * (255 - offset)) + offset
    g = offset - int(out_d.get() * (offset))
    b = offset - int(out_d.get() * (offset))
    out_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_e(event):
    r = int(out_e.get() * (255 - offset)) + offset
    g = offset - int(out_e.get() * (offset))
    b = offset - int(out_e.get() * (offset))
    out_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_f(event):
    r = int(out_f.get() * (255 - offset)) + offset
    g = offset - int(out_f.get() * (offset))
    b = offset - int(out_f.get() * (offset))
    out_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_g(event):
    r = int(out_g.get() * (255 - offset)) + offset
    g = offset - int(out_g.get() * (offset))
    b = offset - int(out_g.get() * (offset))
    out_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
                
def default_btn(event):
    train_callback()

def train_callback():
    global tepochs
    epochs = int(entry_epochs.get())
    for i in range(epochs):
        mse = 0.0
        mse += sdrnn.bp([1,1,1,1,1,1,0],[1,1,1,1,1,1,0])    #0 pattern
        mse += sdrnn.bp([0,1,1,0,0,0,0],[0,1,1,0,0,0,0])    #1 pattern
        mse += sdrnn.bp([1,1,0,1,1,0,1],[1,1,0,1,1,0,1])    #2 pattern
        mse += sdrnn.bp([1,1,1,1,0,0,1],[1,1,1,1,0,0,1])    #3 pattern
        mse += sdrnn.bp([0,1,1,0,0,1,1],[0,1,1,0,0,1,1])    #4 pattern
        mse += sdrnn.bp([1,0,1,1,0,1,1],[1,0,1,1,0,1,1])    #5 pattern
        mse += sdrnn.bp([1,0,1,1,1,1,1],[1,0,1,1,1,1,1])    #6 pattern
        mse += sdrnn.bp([1,1,1,0,0,0,0],[1,1,1,0,0,0,0])    #7 pattern
        mse += sdrnn.bp([1,1,1,1,1,1,1],[1,1,1,1,1,1,1])    #8 pattern
        mse += sdrnn.bp([1,1,1,1,0,1,1],[1,1,1,1,0,1,1])    #9 pattern
    lbl_err.configure(text="{0:.10f}".format(mse/10.0))
    tepochs += epochs
    lbl_tepochs.configure(text = tepochs)
    run_ann()

def run_ann():
    x = []
    x.append(slider_a.get())
    x.append(slider_b.get())
    x.append(slider_c.get())
    x.append(slider_d.get())
    x.append(slider_e.get())
    x.append(slider_f.get())
    x.append(slider_g.get())    
    theoutput = sdrnn.run(x)
    out_a.set(theoutput[0])
    out_b.set(theoutput[1])
    out_c.set(theoutput[2])
    out_d.set(theoutput[3])
    out_e.set(theoutput[4])
    out_f.set(theoutput[5])
    out_g.set(theoutput[6])

def start_GUI():
    global offset 
    global root
    global slider_a
    global slider_b
    global slider_c
    global slider_d
    global slider_e
    global slider_f
    global slider_g
    global lbl_err
    global out_a
    global out_b
    global out_c
    global out_d
    global out_e
    global out_f
    global out_g
    global btn_go
    global entry_epochs
    global lbl_tepochs 
    global tepochs
    tepochs = 0
    offset = 240

    # Create the root object
    root = tk.Tk()
    root.configure(background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    root.title("7 to 7 SDRNN")
    root.resizable(0,0)
    
    #Create Panedwindow  
    panedwindow = tk.PanedWindow(root, orient='horizontal',
    background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))  
    panedwindow.pack(fill="both", expand=True)  
    #Create Frames    
    frame1=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    frame2=tk.Frame(panedwindow,width=400,height=400, relief="sunken", bd=3)  
    frame3=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    # Create Scale Widgets
    slider_a = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_b = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_c = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_d = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_e = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_f = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_g = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    # Create Scale Widgets
    out_a = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_b = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_c = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_d = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_e = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_f = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_g = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
       
    entry_epochs = tk.Entry(frame2, bd=3, width=10)
    entry_epochs.insert(tk.INSERT, "10")
    entry_epochs.bind('<Return>', default_btn)
    lbl_epochs = tk.Label(frame2, height=2, text="Epochs to Train:")

    lbl_tepochs = tk.Label(frame2, height=2, width=20)
    lbl_tepochs_txt = tk.Label(frame2, height=2, text="Epochs so far:")

    lbl_err = tk.Label(frame2, height=2, width=20)
    lbl_err_txt = tk.Label(frame2, height=2, text="Training Error:")
    
    btn_go = tk.Button(frame2, text="Train some more", command = train_callback)
     
    # Use the grid geometry manager to put the widgets in the respective position
    slider_a.grid(row=0, column=1)
    slider_b.grid(row=1, column=2)
    slider_c.grid(row=3, column=2)
    slider_d.grid(row=4, column=1)
    slider_e.grid(row=3, column=0)
    slider_f.grid(row=1, column=0)
    slider_g.grid(row=2, column=1)

    lbl_epochs.grid(      row=0, column=0)
    entry_epochs.grid(    row=0, column=1)
    btn_go.grid(          row=1, column=1)
    lbl_err.grid(         row=2, column=1)
    lbl_err_txt.grid(     row=2, column=0)
    lbl_tepochs_txt.grid( row=3, column=0)
    lbl_tepochs.grid(     row=3, column=1)

    out_a.grid(row=0, column=1)
    out_b.grid(row=1, column=2)
    out_c.grid(row=3, column=2)
    out_d.grid(row=4, column=1)
    out_e.grid(row=3, column=0)
    out_f.grid(row=1, column=0)
    out_g.grid(row=2, column=1)

    panedwindow.add(frame1)  
    panedwindow.add(frame2) 
    panedwindow.add(frame3) 
    panedwindow.paneconfig(frame1,minsize=200)
    panedwindow.paneconfig(frame2,minsize=250)
    panedwindow.paneconfig(frame3,minsize=200)

    lbl_err.configure(text="---")
    lbl_tepochs.configure(text = "0")
    run_ann()
    # The application mainloop
    tk.mainloop()

start_GUI()